In [1]:
import numpy as _np
import spinmob as _s
import matplotlib.pyplot as _plt
import os as _os
from scipy.optimize import curve_fit

In [2]:
block_sizes= []
block_size = 1
ii=0
while ii<23:
    if int(50e3)%block_size==0:
        block_sizes.append(block_size)
        block_size+=1
        ii+=1
    else:
        block_size += 1

In [ ]:
class data_file(object):
    def __init__(N, epsilon, beta, self):
        self.N = N
        self.epsilon = epsilon
        self.beta = beta

In [85]:
def get_data(ep_string):
    current_folder = _os.getcwd()
    N = int(round(float(input("N? "))))
    beta = float(input("beta? "))
    epsilon = float(input("epsilon? "))
    data  = _s.data.load()
    es = _np.array(data['e_tidle'])
    ns = _np.array(data['n'])
    return ns ,es

def exp_decay(x, gamma, amp):
    return amp*_np.exp(-x/gamma)

def correlation(data, data_average):
    data_correlations = []
    correlation_distance = _np.array(range(len(data)-1))+1
    for distance in correlation_distance:
        correlations = data[:-distance]*data[distance::]-data_average**2
        data_correlations.append(_np.average(correlations))
    return data_correlations, correlation_distance

def fit_exp_decay(data, distacnes, p0):
    popt, cov = curve_fit(exp_decay, distacnes[:1000], data[:1000], p0=p0)
    return [popt, cov]

def plot_exp_decay_fit(correlation, distances, popt, ep_str, n_or_e):
    _plt.plot(distances, correlation, label="data")
    _plt.axhline(0, linestyle="--", color="grey", alpha=0.5)
    _plt.plot(distances[:1000], exp_decay(distances[:1000], *popt), label="fit")
    _plt.title(n_or_e+" correlation for ε="+ep_str, size=35)
    _plt.ylabel(r'$\tilde{e_n}\tilde{e}_{n+d} - \langle \tilde{e}\rangle^2$', size=35)
    _plt.xlabel("d", size=35)
    _plt.tick_params(axis="both", labelsize=30)
    _plt.legend(fontsize=30)
    _plt.show()

def choose_unc(es, es_popt, ns, ns_popt):
    es_uncs = []
    ns_uncs = []
    num_trials = len(es)
    for position, block in enumerate(block_sizes):
        blocked_es = _np.reshape(es, (int(num_trials/block), block))
        block_avg_es = _np.average(blocked_es, axis=1)
        unc = _np.std(block_avg_es)/_np.sqrt(len(block_avg_es))
        es_uncs.append(unc)

        blocked_ns = _np.reshape(ns, (int(num_trials/block), block))
        block_avg_ns = _np.average(blocked_ns, axis=1)
        unc = _np.std(block_avg_ns)/_np.sqrt(len(block_avg_ns))
        ns_uncs.append(unc)

        print("E Unc: ", es_uncs[-1], "E correlation/unc: ", _np.sqrt(exp_decay(block, *es_popt))/unc)
        print("n Unc: ", ns_uncs[-1], "n correlation/unc: ", _np.sqrt(exp_decay(block, *ns_popt))/unc)
        print(" num blocks: ", len(block_avg_es), " blocksize: ", block, "choice number", position)
        print("\r\r\r")
    which_block = int(input("Which block_size?"))
    return [block_sizes[which_block], es_uncs[which_block], ns_uncs[which_block]]


In [4]:
used_block_sizes = []
es_avg_across_epsilons = []
es_unc_across_epsilons = []
ns_avg_across_epislons = []
ns_unc_across_epsilons = []
epsilons_used = []

In [41]:

epsilon_strings = ["0.03", "0.02", "0.01", "0.008", "0.005","0.002", "0.001"]
ep_str = epsilon_strings[6]
ns, es = get_data(ep_str)
num_trials = len(es)
trials = _np.array(range(num_trials))
es_average = _np.average(es)
ns_average = _np.average(ns)
print("check 1")
es_correlation, es_distacnes = correlation(es, es_average)
print("check 2")
ns_correlation, ns_distances = correlation(ns, ns_average)
print("check 3")



check 1
check 2
check 3


In [87]:
p0 = [1, 5]
es_popt, es_cov = fit_exp_decay(es_correlation, es_distacnes, p0)
plot_exp_decay_fit(es_correlation, es_distacnes, es_popt, ep_str, r"$\tilde{e}$")

In [43]:
p0 = [1,5]
ns_popt, ns_cov = fit_exp_decay(ns_correlation, ns_distances, p0)
plot_exp_decay_fit(ns_correlation, ns_distances, ns_popt, ep_str, "n")

In [44]:
block_size, es_unc, ns_unc = choose_unc(es, es_popt, ns, ns_popt)
used_block_sizes.append(block_size)
es_avg_across_epsilons.append(es_average)
es_unc_across_epsilons.append(es_unc)
ns_avg_across_epislons.append(ns_average)
ns_unc_across_epsilons.append(ns_unc)
epsilons_used.append(float(ep_str))

E Unc:  0.0015763056099627382 E correlation/unc:  135.14540537261095
n Unc:  0.0033851110377159954 n correlation/unc:  285.359586447528
 num blocks:  100000  blocksize:  1 choice number 0

E Unc:  0.002140252030018895 E correlation/unc:  92.06208191104464
n Unc:  0.0045662077783794885 n correlation/unc:  191.86096759975743
 num blocks:  50000  blocksize:  2 choice number 1

E Unc:  0.0028541123846127715 E correlation/unc:  58.82980160805499
n Unc:  0.006033420464895698 n correlation/unc:  119.43525358989935
 num blocks:  25000  blocksize:  4 choice number 2

E Unc:  0.003108069638859464 E correlation/unc:  49.848330466014204
n Unc:  0.006542935148310733 n correlation/unc:  99.88503506632938
 num blocks:  20000  blocksize:  5 choice number 3

E Unc:  0.0036582940024005735 E correlation/unc:  33.194069345212924
n Unc:  0.007623414043531358 n correlation/unc:  63.95202143925966
 num blocks:  12500  blocksize:  8 choice number 4

E Unc:  0.003912338656098166 E correlation/unc:  26.36002003

In [47]:
print(epsilons_used)
print(es_avg_across_epsilons)
print(es_unc_across_epsilons)
print(ns_avg_across_epislons)
print(ns_unc_across_epsilons)

[0.03, 0.02, 0.01, 0.008, 0.005, 0.002, 0.001]
[0.7244633333333332, 0.7370816666666667, 0.6992883333333334, 0.6799966666666666, 0.6697566666666666, 0.6355149999999998, 0.64827]
[0.0087435309670763, 0.009386704249377294, 0.00968767341572383, 0.010187654020376276, 0.010561949701904263, 0.010642089949751932, 0.01025054162227538]
[1.72278, 1.63402, 1.54489, 1.52134, 1.49264, 1.46153, 1.46068]
[0.0042429630919912555, 0.0046542582438021205, 0.005021906405937889, 0.005199240540694381, 0.0051215735082101475, 0.005174295077592695, 0.005097937700678579]


In [48]:
def cubic_curve(ep, a, b, c, d):
    return a+b*ep+c*ep**2+d*ep**3

In [72]:
e0_p0 = [0.63, 4, -1, 0]
e0_popt, e0_cov = curve_fit(cubic_curve, epsilons_used, es_avg_across_epsilons, p0=e0_p0, sigma=es_unc_across_epsilons)
print(e0_popt)
print(e0_cov)

[ 6.34866589e-01  5.84810497e+00  8.57882499e+01 -6.04375312e+03]
[[ 8.01044461e-05 -2.23733847e-02  1.53751956e+00 -2.94443573e+01]
 [-2.23733847e-02  8.78735860e+00 -6.85632461e+02  1.39862589e+04]
 [ 1.53751956e+00 -6.85632461e+02  5.81083645e+04 -1.24154163e+06]
 [-2.94443573e+01  1.39862589e+04 -1.24154163e+06  2.72532467e+07]]


In [93]:
_plt.errorbar(epsilons_used, es_avg_across_epsilons, yerr=es_unc_across_epsilons, linestyle="", marker=".", label="data")
epsilon_range = _np.linspace(0, max(epsilons_used), 100)
_plt.plot(epsilon_range, cubic_curve(epsilon_range, *e0_popt), label="fit")
_plt.title(r'$\langle \tilde{e} \rangle \: vs \: \epsilon$', fontsize=20)
_plt.xlabel(r'$\epsilon$', fontsize=20)
_plt.ylabel(r'$\langle \tilde{e} \rangle$', fontsize=20)
_plt.legend()

In [88]:
_np.sqrt(e0_cov[0,0])

0.008950108718083357